In [1]:
import sys
sys.path.append('../..')

#from src.creator.tanaka.main import Creator
from src.coder.main import Coder
from src.evaluator.main import Evaluator

llm_name = "gpt-4-turbo-preview"
#llm_name = "gpt-3.5-turbo-0125"
#llm_name = "gemini-pro"
#llm_name = "claude-3-opus-20240229"

### 初めに入力されるモデル

In [7]:
base_code = """
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train(trainloader, params, device):
    net = Net()
    net.to(device)  # Move the model to the specified device

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=params['lr'])

    for epoch in range(1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)  # Move inputs and labels to the specified device

            optimizer.zero_grad()  # zero the parameter gradients

            outputs = net(inputs)  # forward + backward + optimize
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                running_loss = 0.0

    return net

def test(net, testloader, device):
    all_outputs = []

    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)  # Move images to the specified device
            outputs = net(images)
            probabilities = F.softmax(outputs, dim=1)
            probabilities_np = probabilities.cpu().numpy().tolist()  # Move the probabilities back to CPU for numpy conversion
            all_outputs.extend(probabilities_np)

    all_outputs_np = np.array(all_outputs)
    return all_outputs_np

def model(trainloader, testloader, params):
    device = torch.device("cuda")# if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        print("CUDA (GPU support) is available in this environment.")
    net = train(trainloader, params, device)
    y_pred = test(net, testloader, device)
    return y_pred
"""

with open("/Users/tanakatouma/vscode/Mockup_python/data/base_model.py", "w") as file:
    file.write(base_code)

## Creator

* Creatorの出力として欲しい形式

In [3]:
new_method = """
<ELEMENTAL_METHOD name="Momentum Calculation and Application">
This method involves computing the moving average of the gradients (also known as the first moment) and using this to update the model parameters. It's a technique used to accelerate gradients vectors in the right directions, thus leading to faster converging. It is widely used in optimization algorithms, including as a component of the Adam optimizer.

<PYTHON>
def momentum_update(lr, beta1, params, objective, weight_decay, epsilon):
    m = 0  # first moment
    theta = params
    while True:
        g = np.gradient(objective(theta), theta)  # gradient of objective function
        if weight_decay != 0:
            g += weight_decay * theta
        m = beta1 * m + (1 - beta1) * g
        theta -= lr * m
        yield theta
</PYTHON>
</ELEMENTAL_METHOD>
"""


with open("/Users/tanakatouma/vscode/Mockup_python/data/new_method.txt", "w") as file:
    file.write(new_method)

## Coder

In [4]:
coder = Coder(llm_name = llm_name)

In [5]:
base_file_path = "/Users/tanakatouma/vscode/Mockup_python/data/base_model.py"
new_method_path = "/Users/tanakatouma/vscode/Mockup_python/data/new_method.txt"
save_file_path="/Users/tanakatouma/vscode/Mockup_python/data/exec_code.py"

exec_code_path = coder.exec(base_file_path, new_method_path, save_file_path)

In [6]:
exec_code_path

'/Users/tanakatouma/vscode/Mockup_python/data/exec_code.py'

## Evaluator

In [6]:
params = {
    'lr': {'type': 'log_float', 'args': [1e-8, 0.1]},
    'iterations': {'type': 'log_float', 'args': [100, 1000]},
    "lambda": 1
}

In [7]:
eval = Evaluator(
    llm_name=llm_name,
    dataset_name='cifar10',
    params=params,
    valuation_index='accuracy',
    datasave_path='../../data',
    n_trials=10,
    )

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
result = eval.exec(exec_code_path)

## 実際にループを回す場合

In [ ]:

llm_name = "gpt-4-turbo-preview"
#llm_name = "gpt-3.5-turbo-0125"
#llm_name = "gemini-pro"
#llm_name = "claude-3-opus-20240229"


#creator = Creator(llm_name=llm_name)

coder = Coder(
    llm_name = llm_name
    )

eval = Evaluator(
    llm_name=llm_name,
    dataset_name='cifar10',
    params=params,
    valuation_index='accuracy',
    datasave_path='../../data',
    n_trials=10,
    )

In [ ]:
# 一番初めにベースモデルの評価から行うためevaluatorから実行
# 10回の改善を行う

for i in range(10):
    file_name = f'exec_code_{i}.py'
    result = eval.exec(exec_code_path)
    base_code_path = exec_code_path
    if i == 9:
        break
    #new_method_path = creator.exec(base_code, result)
    exec_code_path = coder.exec(base_code_path, new_method_path, file_name)